In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, max, min

In [2]:
# Initialize Spark session
spark = SparkSession.builder.appName("BigData_Car_Analysis").getOrCreate()

In [3]:
# Load dataset
file_path = "car_dataset_india.csv"  
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [4]:
# Show schema and first few rows
df.printSchema()
df.show(5)

root
 |-- Car_ID: integer (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Fuel_Type: string (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Mileage: double (nullable = true)
 |-- Engine_CC: integer (nullable = true)
 |-- Seating_Capacity: integer (nullable = true)
 |-- Service_Cost: double (nullable = true)

+------+-------------+------+----+---------+------------+---------+-------+---------+----------------+------------+
|Car_ID|        Brand| Model|Year|Fuel_Type|Transmission|    Price|Mileage|Engine_CC|Seating_Capacity|Service_Cost|
+------+-------------+------+----+---------+------------+---------+-------+---------+----------------+------------+
|     1|       Toyota|Innova|2024|      CNG|      Manual|2020000.0|   27.3|      800|               4|     24100.0|
|     2|          Kia|   EV6|2023|   Diesel|      Manual|1770000.0|   16.4|     2500

In [5]:
# Check for missing values
print("Missing Values:")
df.select([count(col(c)).alias(c) for c in df.columns]).show()

Missing Values:
+------+-----+-----+-----+---------+------------+-----+-------+---------+----------------+------------+
|Car_ID|Brand|Model| Year|Fuel_Type|Transmission|Price|Mileage|Engine_CC|Seating_Capacity|Service_Cost|
+------+-----+-----+-----+---------+------------+-----+-------+---------+----------------+------------+
| 10000|10000|10000|10000|    10000|       10000|10000|  10000|    10000|           10000|       10000|
+------+-----+-----+-----+---------+------------+-----+-------+---------+----------------+------------+



In [6]:
# Drop rows with missing values
df = df.dropna()

In [7]:
# Exploratory Data Analysis (EDA)
print("Basic Statistics:")
df.describe().show()

Basic Statistics:
+-------+------------------+----------+------+-----------------+---------+------------+-----------------+-----------------+-----------------+------------------+-----------------+
|summary|            Car_ID|     Brand| Model|             Year|Fuel_Type|Transmission|            Price|          Mileage|        Engine_CC|  Seating_Capacity|     Service_Cost|
+-------+------------------+----------+------+-----------------+---------+------------+-----------------+-----------------+-----------------+------------------+-----------------+
|  count|             10000|     10000| 10000|            10000|    10000|       10000|            10000|            10000|            10000|             10000|            10000|
|   mean|            5000.5|      NULL|  NULL|        2019.5438|     NULL|        NULL|        1946064.0|19.96730000000002|          1542.07|            5.5154|         14969.13|
| stddev|2886.8956799071675|      NULL|  NULL|2.877552708632654|     NULL|        NULL|

In [8]:
# Aggregation: Average Price by Fuel Type
fuel_price_avg = df.groupBy("Fuel_Type").agg(avg("Price").alias("Avg_Price"))
fuel_price_avg.show()

+---------+------------------+
|Fuel_Type|         Avg_Price|
+---------+------------------+
|   Diesel| 1945530.794165316|
|      CNG|1934733.3848531684|
| Electric|1951938.0460683082|
|   Petrol|1952596.8672712285|
+---------+------------------+



In [9]:
# Aggregation: Maximum and Minimum Price by Brand
brand_price_stats = df.groupBy("Brand").agg(
    max("Price").alias("Max_Price"),
    min("Price").alias("Min_Price")
)
brand_price_stats.show()

+-------------+---------+---------+
|        Brand|Max_Price|Min_Price|
+-------------+---------+---------+
|   Volkswagen|3500000.0| 400000.0|
|          Kia|3500000.0| 400000.0|
|      Hyundai|3500000.0| 400000.0|
|        Honda|3490000.0| 400000.0|
|  Tata Motors|3490000.0| 400000.0|
|      Renault|3500000.0| 410000.0|
|     Mahindra|3490000.0| 410000.0|
|Maruti Suzuki|3500000.0| 400000.0|
|        Skoda|3500000.0| 400000.0|
|       Toyota|3500000.0| 400000.0|
+-------------+---------+---------+



In [10]:
# Top 5 Expensive Cars
top_expensive = df.orderBy(col("Price").desc()).limit(5)
top_expensive.show()

+------+-------------+------+----+---------+------------+---------+-------+---------+----------------+------------+
|Car_ID|        Brand| Model|Year|Fuel_Type|Transmission|    Price|Mileage|Engine_CC|Seating_Capacity|Service_Cost|
+------+-------------+------+----+---------+------------+---------+-------+---------+----------------+------------+
|  1153|      Hyundai| Venue|2015|      CNG|      Manual|3500000.0|   14.3|     1200|               5|     13800.0|
|  4044|      Renault|  Kwid|2022|   Petrol|      Manual|3500000.0|   11.9|     1200|               4|     19900.0|
|  2594|      Renault| Lodgy|2020|      CNG|   Automatic|3500000.0|   27.2|     1500|               6|      8600.0|
|  3157|        Skoda| Rapid|2022|      CNG|   Automatic|3500000.0|   13.9|     1500|               4|     19500.0|
|   901|Maruti Suzuki|Ertiga|2023| Electric|   Automatic|3500000.0|   13.0|     1200|               5|     11500.0|
+------+-------------+------+----+---------+------------+---------+-----

In [11]:
# Stop Spark session
spark.stop()